In [2]:
import numpy as np
import pandas as pd

In [4]:
(lambda x: x*2)(12)

24

In [7]:
list_1 = [1,2,3,4,5,6,7,8,9]
filter(lambda x: x%2==0, list_1)

[2, 4, 6, 8]

In [8]:
list(filter(lambda x: x%2==0, list_1))

[2, 4, 6, 8]

In [9]:
list_1 = [1,2,3,4,5,6,7,8,9]
cubed = map(lambda x: pow(x,3), list_1)
list(cubed)

[1, 8, 27, 64, 125, 216, 343, 512, 729]

In [11]:
df = pd.DataFrame({
    'Name': ['Luke','Gina','Sam','Emma'],
    'Status': ['Father', 'Mother', 'Son', 'Daughter'],
    'Birthyear': [1976, 1984, 2013, 2016],
})

df.head()

,Name,Status,Birthyear
0,Luke,Father,1976
1,Gina,Mother,1984
2,Sam,Son,2013
3,Emma,Daughter,2016


In [12]:
df['age'] = df['Birthyear'].apply(lambda x: 2021-x)

In [16]:
df.age.head

<bound method NDFrame.head of 0    45
1    37
2     8
3     5
Name: age, dtype: int64>

In [17]:
# list(filter(lambda x: x>18, df['age']))
list(filter(lambda x: x> 18, df.age))

[45, 37]

In [18]:
df['double_age'] = df['age'].map(lambda x: x*2)

In [21]:
df['Gender'] = df['Status'].map(lambda x: 'Male' if x=='Father' or x=='son' else 'Female')

In [22]:
df.head()

,Name,Status,Birthyear,age,double_age,Gender
0,Luke,Father,1976,45,90,Male
1,Gina,Mother,1984,37,74,Female
2,Sam,Son,2013,8,16,Female
3,Emma,Daughter,2016,5,10,Female


In [23]:
import re

def putSpace(input):

    # regex [A-Z][a-z]* means any string starting
    # with capital character followed by many
    # lowercase letters
    words = re.findall('[A-Z][a-z]*', input)

    print(words)

    # Change first letter of each word into lower
    # case
    for i in range(0,len(words)):
        words[i]=words[i][0].lower()+words[i][1:]

    print(words)
    print(' '.join(words))

# Driver program
if __name__ == "__main__":
    input = 'BruceWayneIsBatman'
    putSpace(input)

['Bruce', 'Wayne', 'Is', 'Batman']
['bruce', 'wayne', 'is', 'batman']
bruce wayne is batman


In [24]:
text_try = "digrebeg di hotel, aktor ftv ridho illahi berduaan dengan selebgram chagii wow! amelia?"

In [26]:
df['Status'] = df['Status'].replace({'.':' . ',', ':' , ', '!': " ! ", "?": " ? "},regex=True)

In [34]:
text_try = text_try.replace(".", " . " )

res_text = text_try.replace(",", " , " )

# res_text = text_try.replace({'.':' . ',', ':' , ', '!': " ! ", "?": " ? "}, )

print(res_text)

digrebeg di hotel ,  aktor ftv ridho illahi berduaan dengan selebgram chagii amelia?


In [41]:
rule = "([\D])([^\w^\s])([\s$\D])?"

text = "sering quality time bersama keluarga? ternyata ini 3,000 man,3 seperti manfaatnya!"

clean = re.compile(rule)
output = re.sub(clean, '\1 \2 \3' , text)

print(output)
# res = text.apply(lambda y: " ".join((re.sub(r'([!/?/./,/%/\'/\"/-/:])', lambda x: ' ' + x.group()+' ' , y)).split()) , 1)

sering quality time bersama keluarg  ternyata ini 3,000 ma  3 seperti manfaatny  
